In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
np.random.seed(1)

ModuleNotFoundError: No module named 'tkinter'

In [ ]:
# read data
df = pd.read_csv('data/preprocessed_data.csv')
df

,SSL SCORE,PREDICTOR RAT AGE AT LATEST ARREST,PREDICTOR RAT VICTIM SHOOTING INCIDENTS,PREDICTOR RAT VICTIM BATTERY OR ASSAULT,PREDICTOR RAT ARRESTS VIOLENT OFFENSES,PREDICTOR RAT GANG AFFILIATION,PREDICTOR RAT NARCOTIC ARRESTS,PREDICTOR RAT TREND IN CRIMINAL ACTIVITY,PREDICTOR RAT UUW ARRESTS,SEX CODE CD,...,NARCOTICS ARR CNT,CPD ARREST I,DOMESTIC ARR CNT,AGE CURR,RAW SSL SCORE,HEAT SCORE,RAW_HEAT_SCORE,STATUS I,PRE RAW HEAT SCORE,COMMUNITY AREA
0,306,20-30,0,0,0,0,0,-0.3,0,M,...,0.0,N,0.0,20-30,0.09110,306,0.09110,Y,9110,
1,352,20-30,0,1,0,0,0,0.0,0,M,...,0.0,N,0.0,20-30,0.09949,352,0.09949,Y,9949,
2,349,20-30,0,0,2,0,2,0.0,0,M,...,0.0,N,0.0,20-30,0.09908,349,0.09908,Y,9908,
3,344,20-30,0,0,3,0,0,-0.3,0,M,...,0.0,N,0.0,20-30,0.09810,344,0.09810,Y,9810,
4,343,20-30,0,0,1,0,0,0.4,0,M,...,0.0,N,0.0,20-30,0.09794,343,0.09794,Y,9794,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398679,183,50-60,0,0,0,0,0,0.3,0,F,...,0.0,Y,0.0,50-60,0.06845,183,0.06845,Y,6845,NEW CITY
398680,213,60-70,0,0,2,0,0,2.2,0,M,...,0.0,Y,0.0,60-70,0.07396,213,0.07396,Y,7396,OHARE
398681,277,30-40,0,0,0,0,0,-0.3,0,M,...,0.0,Y,0.0,30-40,0.08569,277,0.08569,Y,8569,NEW CITY
398682,220,40-50,0,0,0,0,0,-0.3,0,F,...,0.0,Y,0.0,40-50,0.07535,220,0.07535,Y,7535,GARFIELD RIDGE


In [ ]:
# define subsets to look at sex, race, age, weapons, and drugs
df_onehot = pd.concat([df['SSL SCORE'], pd.get_dummies(df['SEX CODE CD', 'RACE CODE CD', 'AGE GROUP'])], axis=1)
df_onehot

,SSL SCORE,SEX CODE CD_F,SEX CODE CD_M,SEX CODE CD_X,RACE CODE CD_API,RACE CODE CD_BLK,RACE CODE CD_I,RACE CODE CD_U,RACE CODE CD_WBH,RACE CODE CD_WHI,RACE CODE CD_WWH,AGE GROUP_20-30,AGE GROUP_30-40,AGE GROUP_40-50,AGE GROUP_50-60,AGE GROUP_60-70,AGE GROUP_70-80,AGE GROUP_less than 20
0,306,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
1,352,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,349,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
3,344,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,343,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398679,183,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
398680,213,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
398681,277,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
398682,220,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [ ]:
df_aif = BinaryLabelDataset(df=df_onehot.drop(axis=1), label_names=['SSL SCORE'], protected_attribute_names=['SEX CODE CD_F'])

NameError: name 'BinaryLabelDataset' is not defined

In [ ]:
def get_disparity_index(di):
    return 1 - np.minimum(di, 1 / di)


def get_bal_acc(classified_metric):
    return 0.5 * (classified_metric.true_positive_rate() + classified_metric.true_negative_rate())


def train_lr_model(dataset):
    scale = StandardScaler().fit(dataset.features)
    model = LogisticRegression(random_state=0, solver='liblinear')
    x_train = scale.transform(dataset.features)
    y_train = dataset.labels.ravel()
    model.fit(x_train, y_train, sample_weight=dataset.instance_weights)
    y_train_pred = model.predict(x_train)
    return model, scale


def test_lr_model(y_data_pred_prob, dataset, thresh_arr):
    y_pred = (y_data_pred_prob[:,1] > thresh_arr).astype(np.double)
    dataset_pred = dataset.copy()
    dataset_pred.labels = y_pred
    classified_metric = ClassificationMetric(dataset, dataset_pred, unprivileged_group, privileged_group)
    metric_pred = BinaryLabelDatasetMetric(dataset_pred, unprivileged_group, privileged_group)
    return dataset_pred.labels, classified_metric, metric_pred


def get_y_pred_prob_lr(scale, model, dataset):
    x = scale.transform(dataset.features)
    y_pred_prob = model.predict_proba(x)
    return y_pred_prob


def get_best_bal_acc_cutoff(y_pred_prob, dataset):
    y_validate_pred_prob = y_pred_prob
    bal_acc_arr = []
    disp_imp_arr = []

    for thresh in tqdm(thresh_arr):
        y_validate_pred = (y_validate_pred_prob[:,1] > thresh).astype(np.double)
        dataset_pred = dataset.copy()
        dataset_pred.labels = y_validate_pred

        # Calculate accuracy for each threshold value
        classified_metric = ClassificationMetric(dataset, dataset_pred, unprivileged_group, privileged_group)
        bal_acc = get_bal_acc(classified_metric)
        bal_acc_arr.append(bal_acc)

        # Calculate fairness for each threshold value
        metric_pred = BinaryLabelDatasetMetric(dataset_pred, unprivileged_group, privileged_group)
        disp_imp_arr.append(metric_pred.disparate_impact())

    # Find threshold for best accuracy
    thresh_arr_best_ind = np.where(bal_acc_arr == np.max(bal_acc_arr))[0][0]
    thresh_arr_best = np.array(thresh_arr)[thresh_arr_best_ind]

    # Calculate accuracy and fairness at this threshold
    best_bal_acc = bal_acc_arr[thresh_arr_best_ind]
    disp_imp_at_best_bal_acc = disp_imp_arr[thresh_arr_best_ind]

    # Output metrics
    acc_metrics = pd.DataFrame({'thresh_arr_best_ind' : thresh_arr_best_ind, \
    'thresh_arr_best' : thresh_arr_best, \
    'best_bal_acc' : best_bal_acc, \
    'disp_imp_at_best_bal_acc' : disp_imp_at_best_bal_acc}, index=[0]).transpose()
    return acc_metrics, bal_acc_arr, disp_imp_arr, dataset_pred.labels


def plot_acc_vs_fairness(metric, metric_name, bal_acc_arr, thresh_arr_best_ind):
    fig, ax1 = plt.subplots(figsize=(10, 7))
    ax1.plot(thresh_arr, bal_acc_arr, color='b')
    ax1.set_xlabel('Classification Thresholds', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Balanced Accuracy', color='b', fontsize=16, fontweight='bold')
    ax1.xaxis.set_tick_params(labelsize=14)
    ax1.yaxis.set_tick_params(labelsize=14, labelcolor='b')
    ax2 = ax1.twinx()
    ax2.plot(thresh_arr, metric, color='r')
    ax2.set_ylabel(metric_name, color='r', fontsize=16, fontweight='bold')
    ax2.axvline(np.array(thresh_arr)[thresh_arr_best_ind], color='k', linestyle=':')
    ax2.yaxis.set_tick_params(labelsize=14, labelcolor='r')
    ax2.grid(True)

NameError: name 'aif360' is not defined